<a href="https://colab.research.google.com/github/ClauCll/Curso_ML_A_Z/blob/master/Regresi%C3%B3n/Regresion_Bajas_Antiguedad.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Cuantas serán las bajas voluntarias de los clientes fijos para el periodo 202009. 
 

In [8]:
import pandas as pd
import numpy as np
from mpl_toolkits.mplot3d import axes3d
import matplotlib.pyplot as plt
import gspread
from oauth2client.client import GoogleCredentials 
from google.colab import auth
# Autoriza Claudia
auth.authenticate_user()
gc = gspread.authorize (GoogleCredentials.get_application_default())

### leemos datos

In [9]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [10]:
data1 = pd.read_csv('/content/drive/My Drive/Dataset/DBajas_ANTIGUEDAD.txt',delimiter='~')
x = data1.iloc [:,:-1].values
y = data1.iloc [:,2].values

In [ ]:
x

### Codificar datos categóricos

In [7]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer
labelencoder  = LabelEncoder()
labelencoder  = LabelEncoder()
x[:,1] = labelencoder.fit_transform(x[:,1])
ct = ColumnTransformer([('one_hot_encoder',OneHotEncoder(categories='auto'),[1])],remainder='passthrough')
x= np.array(ct.fit_transform(x),dtype=np.integer)

ValueError: ignored

In [ ]:
pd.DataFrame(x)

### Evitar la trampa de variables Dummy

In [ ]:
'''x = x[:,1:]'''

### Dividimos los datos en entrenamiento y test

In [11]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size= 0.2, random_state=0)

### Escalamos los datos

In [224]:
'''from sklearn.preprocessing import StandardScaler
sc_x = StandardScaler()
sc_y = StandardScaler()
x_train= sc_x.fit_transform(x_train)
y_train= sc_y.fit_transform(y_train.reshape(-1,1))'''

In [ ]:
'''len(x_train[:,0])

In [ ]:
'''pd.DataFrame(x_train)

In [12]:
x_train

array([[202007, '1_3_ANIOS'],
       [202003, '4_TRIM'],
       [202002, '4_TRIM'],
       [201909, '2_TRIM'],
       [201908, '3_TRIM'],
       [202004, '4_6_ANIOS'],
       [201909, '4_6_ANIOS'],
       [202006, '1_3_ANIOS'],
       [202003, '1_TRIM'],
       [202004, '2_TRIM'],
       [201912, '4_6_ANIOS'],
       [202002, '1_3_ANIOS'],
       [202008, 'MAYOR_6_ANIOS'],
       [201908, '2_TRIM'],
       [202005, '3_TRIM'],
       [201910, '4_TRIM'],
       [202005, '4_TRIM'],
       [202007, '4_6_ANIOS'],
       [202005, '2_TRIM'],
       [202001, '4_TRIM'],
       [201912, '4_TRIM'],
       [201909, '3_TRIM'],
       [202005, '4_6_ANIOS'],
       [201911, '1_3_ANIOS'],
       [201909, '4_TRIM'],
       [202001, '1_3_ANIOS'],
       [202007, 'MAYOR_6_ANIOS'],
       [202007, '4_TRIM'],
       [201908, '4_TRIM'],
       [202001, '3_TRIM'],
       [202007, '3_TRIM'],
       [201910, '1_TRIM'],
       [202004, '1_3_ANIOS'],
       [201911, '1_TRIM'],
       [201912, '2_TRIM'],
       [

### DETERMINAR DIMENSIONES.  ELIMINACION HACIA ATRAS

In [ ]:
##x_train=np.append(arr=np.ones((len(x_train[:,0]),1)).astype(int),values=x_train,axis=1)

In [ ]:
##pd.DataFrame(x_train)

In [ ]:
'''import statsmodels.api as sm
def elimnaAtras (x,SL):
  numvar = len(x[0])
  for i in range(0 , numvar):
    regresor_ols = sm.OLS(y, x.tolist()).fit()
    maxvar=max(regresor_ols.pvalues).astype(float)
    if maxvar>SL:
      for j in range(0,numvar-i):
        if (regresor_ols.pvalues[j].astype(float)== maxvar):
          x=np.delete(x,j,1)
  regresor_ols.summary()
  return x
SL=0.05
x_opt = x[:,[0,1,2,3,4,5,6,7]]        
x_modelo = elimnaAtras(x_opt, SL)'''

In [ ]:
##pd.DataFrame(x_modelo)

### MODELO REGRESION POLINOMICA

In [271]:
from sklearn.preprocessing import PolynomialFeatures
poly_reg = PolynomialFeatures(degree = 2)
x_poly = poly_reg.fit_transform(x_train)

from sklearn.linear_model import LinearRegression
lineal_poly = LinearRegression()
lineal_poly.fit(x_poly,y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [226]:
'''x_valor=sc_x.inverse_transform(x_train)
y_valor= sc_y.inverse_transform(y_train)
y_predic_valor= sc_y.inverse_transform(regression.predict(x_train))'''

In [ ]:
pd.DataFrame(lineal_poly.predict(x_poly))

In [286]:
pd.DataFrame(x_poly)

,0,1,2,3,4,5
0,1.0,202007.0,7.0,4.080683e+10,1414049.0,49.0
1,1.0,202003.0,3.0,4.080521e+10,606009.0,9.0
2,1.0,202002.0,3.0,4.080481e+10,606006.0,9.0
3,1.0,201909.0,5.0,4.076724e+10,1009545.0,25.0
4,1.0,201908.0,2.0,4.076684e+10,403816.0,4.0
...,...,...,...,...,...,...
60,1.0,202008.0,7.0,4.080723e+10,1414056.0,49.0
61,1.0,202006.0,4.0,4.080642e+10,808024.0,16.0
62,1.0,202006.0,5.0,4.080642e+10,1010030.0,25.0
63,1.0,202003.0,5.0,4.080521e+10,1010015.0,25.0


In [ ]:
pd.DataFrame(y_train)

In [ ]:
# GRAFICO ENTRENAMIENTO
x_grid = np.arange(min(x_train[:,0]), max(x_train[:,0]), 0.1)
x_grid= x_grid.reshape(len(x_grid),1)
plt.scatter (x_train[:,0],y_train, color="Red")
plt.plot (x_train[:,0],lineal_poly.predict(x_poly),color="blue")
plt.title ("Modelo polinómico")
plt.xlabel("Periodo")
plt.ylabel("Cantidad Bajas")
plt.show()

In [ ]:
x_train[:,0]